In [ ]:
import os
import numpy as np
from glob import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split
import cv2
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.layers import Conv2D , BatchNormalization , Activation , MaxPool2D , Input , Dropout , ZeroPadding2D , Conv2DTranspose , Concatenate

from tensorflow.keras.models import Model

from tensorflow.keras.applications import InceptionResNetV2
def conv_block(inputs , num_filters):
  x = Conv2D(num_filters , 3 , padding= "same" )(inputs)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)
  x = Dropout(0.3)(x)

  x = Conv2D(num_filters , 3 , padding= "same" )(x)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)
  x = Dropout(0.3)(x)

  return x


def decoder_block(inputs , skip_features , num_filters):

  x = Conv2DTranspose(num_filters , (2, 2) , strides = 2 , padding = "same")(inputs)
  x = Concatenate()([x , skip_features ])
  x = conv_block(x , num_filters)

  return x


In [ ]:
def build_inception_resnetv2_unet(input_shape):

  #Input
  inputs = Input(input_shape)

### Pre-trained InceptionResNetV2 Model being used in UNet Architecture
  #Encoder
  # We loaded the weights as We want to use Pre-trained Weights, of imagenet, for InceptionResnet
  #We only require CNN Layers and NO FULLY CONNECTED LAYERS , thus include_top is set as False

  encoder = InceptionResNetV2(include_top = False  , weights = "imagenet" , input_tensor = inputs)
  encoder.summary()

  #Skip Connection
  s1 = encoder.get_layer("input_1").output                 #Shape will be 256 * 256

  s2 = encoder.get_layer("activation_2").output         # 125 * 125 
  s2 = ZeroPadding2D( ((2 , 1) ,  ( 2 , 1)) )(s2)          # 128 * 128


  s3 = encoder.get_layer("activation_4").output        # 60 * 60
  s3 = ZeroPadding2D( ((2 , 2 ) , (2, 2)) )(s3)         # 64 * 64 


  s4 = encoder.get_layer("activation_74").output                              # 29*29
  s4 = ZeroPadding2D( ((2 , 1) ,  ( 2 , 1)) )(s4)                              # 32 * 32
  # Padded with 2 on top , 1 in bottom     2 in left and 1 in right

  print(s1.shape , s2.shape , s3.shape ,  s4.shape)
  #Bridge

  b1 = encoder.get_layer("activation_161").output                      # 14 * 14
  b1 = ZeroPadding2D( ((1 , 1) , (1, 1)) )(b1)                         # 16 * 16 

  print(b1.shape)

  ## DECODER

  d1 = decoder_block(b1 , s4 , 512)               # 32 * 32                
  d2 = decoder_block(d1 , s3  , 256)              # 64* 64
  d3 = decoder_block(d2 , s2 , 128)               # 128* 128
  d4 = decoder_block(d3 , s1 , 64)                # 256*256        # Atlast it should reach INPUT SHAPE which is 512 * 512 in our case

  print(d4.shape)


  ## OUTPUT

  outputs = Conv2D( 1 , 1 , padding = "same" , activation = "sigmoid")(d4)

  ## model

  model = Model(inputs , outputs  , name = "InceptionResNetV2_U-Net")
  return model


In [ ]:
if __name__  == "__main__":
  shape = (256 , 256 , 3)
  model = build_inception_resnetv2_unet(shape)
  model.summary()

219062272/219055592 [==============================] - 6s 0us/step
Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 127, 127, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 127, 127, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 127, 127, 32) 0           batch_normalization[0][0]        
_____________

In [ ]:
def load_data(paths , split = 0.15):
  images = []
  masks = []
  numImages = 0
  numMasks = 0

  for path in paths:
    
        
    images.extend(sorted(glob(os.path.join(path , "images/*"))))
    masks.extend(sorted(glob(os.path.join(path , "label/*"))))
    if "training" in path:
      for i in ["00", "01" , "02" , "03" , "04" , "05" , "07", "08" , "09" , "10" , "11" , "12" , "13" , "14" , "16" , "17" , "18" , "19" , "21" , "22" , "24" , "26" , "27" ]:
        images.remove("../input/chagas/ChagasTraining/training-20210525T143718Z-001/training/images/i8{}.xml".format(i))

      n = "training"
    if "Test" in path:
      n= "Test"
    if "Val" in path:

      n= "Validation"

    print("{}PathImagesAre:".format(n),len(images) - numImages)
    print("{}PathImagesAre:".format(n),len(masks) - numMasks)

    numImages = len(images)
    numMasks = len(masks)

  total_size = len(images)
  valid_size = int(split * total_size)
  test_size = int(split * total_size)
  print("Number of images in Total , Validation images and Test Images : " , total_size , valid_size ,test_size  )

  train_x , test_x = train_test_split(images , test_size = test_size , random_state = 42 )
  train_y , test_y = train_test_split(masks , test_size = test_size , random_state = 42 )

  train_x , valid_x = train_test_split(train_x , test_size = valid_size , random_state = 42 )
  train_y , valid_y = train_test_split(train_y , test_size = valid_size , random_state = 42 )

  return (train_x , train_y) , (valid_x , valid_y ) , (test_x , test_y) 

def read_image(path):
  path = path.decode()
  x = cv2.imread(path , cv2.IMREAD_COLOR)
  x = cv2.resize(x , (256 , 256))
  x = x/255.0
  #Size is 256 * 256 * 3

  return x

def read_mask(path):
  path = path.decode()
  x = cv2.imread(path , cv2.IMREAD_GRAYSCALE)
  x = cv2.resize(x , (256 , 256))
  x = x/255.0
  
  #Size is 256 * 256
  x = np.expand_dims(x , axis = -1)
  #Size becames 256 * 256 * 1

  return x

def tf_parse(imagepath , maskpath):
  def _parse(imagepath , maskpath):
    x = read_image(imagepath)
    y = read_mask(maskpath)

    return x , y

  x , y = tf.numpy_function(_parse , [imagepath , maskpath] , [tf.float64 , tf.float64] )
  x.set_shape([256 , 256 , 3])
  y.set_shape([256, 256, 1])

  return x , y 


def tf_dataset( imagepath , maskpath , batch = 8):
  dataset = tf.data.Dataset.from_tensor_slices((imagepath , maskpath))
  dataset = dataset.map(tf_parse)
  dataset = dataset.batch(batch)
  dataset = dataset.repeat()
  return dataset

if __name__ == "__main__":
  paths = ["../input/chagas/ChagasTraining/training-20210525T143718Z-001/training" , "../input/chagas/ChagasTest/Test", "../input/chagas/ChagasValidation/Validation"] 
  (train_x , train_y) , (valid_x , valid_y ) , (test_x , test_y)  =   load_data(paths)

  print("Number of images in Train , Validation images and Test Images : " ,len(train_x) , len(valid_x ) , len(test_x) )

  ds = tf_dataset(test_x , test_y)
  for x, y in ds:
    
    print(x.shape , y.shape)
    break
    


trainingPathImagesAre: 600
trainingPathImagesAre: 600
TestPathImagesAre: 200
TestPathImagesAre: 200
ValidationPathImagesAre: 200
ValidationPathImagesAre: 200
Number of images in Total , Validation images and Test Images :  1000 150 150
Number of images in Train , Validation images and Test Images :  700 150 150
(8, 256, 256, 3) (8, 256, 256, 1)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model





In [ ]:
# Train 
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
from tensorflow.keras.metrics import Recall , Precision 

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)


In [ ]:
pip install segmentation_models

     |████████████████████████████████| 50 kB 1.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%env SM_FRAMEWORK=tf.keras

env: SM_FRAMEWORK=tf.keras


In [ ]:
if __name__ == "__main__":
    from segmentation_models.losses import bce_jaccard_loss
    from segmentation_models.metrics import iou_score
    np.random.seed(42)
    tf.random.set_seed(42)
    #HYPERPARAMETER


    batch = 8
    lr = 1e-4
    epochs = 60
    opt = tf.keras.optimizers.Adam(lr)
    metrics = ["acc" , Recall() , Precision() , iou_score]
    #Data
    train_dataset = tf_dataset(train_x, train_y, batch=batch)
    valid_dataset = tf_dataset(valid_x, valid_y, batch=batch)

    #model = build_model()
    #model.summary()



    model.compile(loss=bce_jaccard_loss, optimizer=opt, metrics=metrics)

    callbacks = [
        ModelCheckpoint("files/model.h5"),
        ReduceLROnPlateau(monitor='val_iou', factor=0.1, patience=4),
        CSVLogger("../data.csv"),
        TensorBoard(),
        EarlyStopping(monitor='val_iou', patience=20, restore_best_weights=False)
    ]
    #Number Of Batches
    train_steps = len(train_x)//batch
    valid_steps = len(valid_x)//batch

    if len(train_x) % batch != 0:
        train_steps += 1
    if len(valid_x) % batch != 0:
        valid_steps += 1

    model.fit(train_dataset,
        validation_data=valid_dataset,
        epochs=epochs,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks,
        shuffle = False)


Segmentation Models: using `tf.keras` framework.
Epoch 1/60
88/88 [==============================] - 67s 466ms/step - loss: 1.6016 - acc: 0.7182 - recall: 0.3024 - precision: 0.0172 - iou_score: 0.0140 - val_loss: 1.5017 - val_acc: 0.9700 - val_recall: 0.3799 - val_precision: 0.2425 - val_iou_score: 0.0158
Epoch 2/60
88/88 [==============================] - 30s 340ms/step - loss: 1.1671 - acc: 0.9835 - recall: 0.3801 - precision: 0.5406 - iou_score: 0.0382 - val_loss: 1.1717 - val_acc: 0.9855 - val_recall: 0.1443 - val_precision: 0.8605 - val_iou_score: 0.0204
Epoch 3/60
88/88 [==============================] - 30s 340ms/step - loss: 1.0777 - acc: 0.9851 - recall: 0.7711 - precision: 0.6593 - iou_score: 0.0771 - val_loss: 1.0601 - val_acc: 0.9861 - val_recall: 0.3527 - val_precision: 0.7762 - val_iou_score: 0.0543
Epoch 4/60
88/88 [==============================] - 30s 342ms/step - loss: 1.0057 - acc: 0.9885 - recall: 0.8260 - precision: 0.7946 - iou_score: 0.1045 - val_loss: 0.9855 - 

In [ ]:
    model.fit(train_dataset,
        validation_data=valid_dataset,
        epochs=30,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks,
        shuffle = False)


Epoch 1/30
88/88 [==============================] - 32s 366ms/step - loss: 0.1334 - acc: 0.9922 - recall: 0.8888 - precision: 0.9888 - iou_score: 0.8756 - val_loss: 0.6049 - val_acc: 0.9883 - val_recall: 0.4639 - val_precision: 0.9369 - val_iou_score: 0.4640
Epoch 2/30
88/88 [==============================] - 30s 343ms/step - loss: 0.1317 - acc: 0.9922 - recall: 0.8899 - precision: 0.9894 - iou_score: 0.8772 - val_loss: 0.4497 - val_acc: 0.9891 - val_recall: 0.6469 - val_precision: 0.8839 - val_iou_score: 0.6046
Epoch 3/30
88/88 [==============================] - 30s 339ms/step - loss: 0.1339 - acc: 0.9922 - recall: 0.8865 - precision: 0.9885 - iou_score: 0.8753 - val_loss: 0.4341 - val_acc: 0.9890 - val_recall: 0.6865 - val_precision: 0.8640 - val_iou_score: 0.6182
Epoch 4/30
88/88 [==============================] - 30s 338ms/step - loss: 0.1362 - acc: 0.9922 - recall: 0.8852 - precision: 0.9880 - iou_score: 0.8732 - val_loss: 0.4305 - val_acc: 0.9888 - val_recall: 0.7098 - val_precis

In [ ]:
import os
  
# Directory
directory = "RESULT_Inception_Resnetv2_WithDropout"
  
# Parent Directory path
parent_dir = "./"
  
# Path
path = os.path.join(parent_dir, directory)
  
# Create the directory
# 'GeeksForGeeks' in
# '/home / User / Documents'
os.mkdir(path)

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from tqdm import tqdm
#from data import load_data, tf_dataset
#from train import iou

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    return x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (256, 256))
    x = np.expand_dims(x, axis=-1)
    return x

def mask_parse(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

if __name__ == "__main__":
    ## Dataset
    
    batch_size = 8
    #(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data("/content/drive/MyDrive/Test")

    test_dataset = tf_dataset(test_x, test_y, batch=batch_size)

    test_steps = (len(test_x)//batch_size)
    if len(test_x) % batch_size != 0:
        test_steps += 1

    #with CustomObjectScope({'iou': iou}):
    #    model = tf.keras.models.load_model("/content/drive/model.h5")

    #model.evaluate(test_dataset, steps=test_steps)

    for i, (x, y) in tqdm(enumerate(zip(test_x, test_y)), total=len(test_x)):
        x = read_image(x)
        y = read_mask(y)
        y_pred = model.predict(np.expand_dims(x, axis=0))[0] > 0.5
        h, w, _ = x.shape
        white_line = np.ones((h, 10, 3)) * 255.0

        all_images = [
            x * 255.0, white_line,
            mask_parse(y), white_line,
            mask_parse(y_pred) * 255.0
        ]
        image = np.concatenate(all_images, axis=1)
        
        cv2.imwrite(f"./RESULT_Inception_Resnetv2_WithDropout/{i}.png", image)

100%|██████████| 150/150 [00:22<00:00,  6.63it/s]
